In [1]:
import pyttsx3
engine = pyttsx3.init()
engine.say("I will speak this text")
engine.save_to_file('Hello World', 'test.mp3')
engine.runAndWait()



'default'

In [13]:
voices = engine.getProperty('voices')
engine.setProperty('voice','english')
engine.say("I will speak this text")
engine.setProperty('voice', 'english+f1')
engine.say("I will speak this text")
engine.runAndWait()



In [44]:
import pyttsx3

engine = pyttsx3.init()
voices = engine.getProperty('voices')
for voice in voices:
    print(voice.id)
engine.setProperty('voice', 'english_rp+f4')
        

engine.say('The quick brown fox jumps over the lazy dog')
engine.runAndWait()

afrikaans
aragonese
bulgarian
bosnian
catalan
czech
welsh
danish
german
greek
default
english
en-scottish
english-north
english_rp
english_wmids
english-us
en-westindies
esperanto
spanish
spanish-latin-am
estonian
persian
persian-pinglish
finnish
french-Belgium
french
irish-gaeilge
greek-ancient
hindi
croatian
hungarian
armenian
armenian-west
indonesian
icelandic
italian
lojban
georgian
kannada
kurdish
latin
lingua_franca_nova
lithuanian
latvian
macedonian
malayalam
malay
nepali
dutch
norwegian
punjabi
polish
brazil
portugal
romanian
russian
slovak
albanian
serbian
swedish
swahili-test
tamil
turkish
vietnam
vietnam_hue
vietnam_sgn
Mandarin
cantonese


# tts with Amazon polly

In [47]:
from boto3 import Session
from botocore.exceptions import BotoCoreError, ClientError
from contextlib import closing
import os
import sys
import subprocess
from tempfile import gettempdir

# Create a client using the credentials and region defined in the [adminuser]
# section of the AWS credentials file (~/.aws/credentials).
session = Session(profile_name="adminuser")
polly = session.client("polly")

try:
    # Request speech synthesis
    response = polly.synthesize_speech(Text="Hello world!", OutputFormat="mp3",
                                        VoiceId="Joanna")
except (BotoCoreError, ClientError) as error:
    # The service returned an error, exit gracefully
    print(error)
    sys.exit(-1)

# Access the audio stream from the response
if "AudioStream" in response:
    # Note: Closing the stream is important because the service throttles on the
    # number of parallel connections. Here we are using contextlib.closing to
    # ensure the close method of the stream object will be called automatically
    # at the end of the with statement's scope.
    with closing(response["AudioStream"]) as stream:
        output = os.path.join(gettempdir(), "speech.mp3")

        try:
            # Open a file for writing the output as a binary stream
            with open(output, "wb") as file:
                file.write(stream.read())
        except IOError as error:
            # Could not write to file, exit gracefully
            print(error)
            sys.exit(-1)

else:
    # The response didn't contain audio data, exit gracefully
    print("Could not stream audio")
    sys.exit(-1)

# Play the audio using the platform's default player
if sys.platform == "win32":
    os.startfile(output)
else:
    # The following works on macOS and Linux. (Darwin = mac, xdg-open = linux).
    opener = "open" if sys.platform == "darwin" else "xdg-open"
    subprocess.call([opener, output])

ProfileNotFound: The config profile (adminuser) could not be found